<a href="https://colab.research.google.com/github/Wpert/numerics/blob/main/numericsInvertible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача:

Для случайных СЛАУ вида $Ax = b$, размерности $n$, найти обратную матрицу, добавив к ней единичную той же размерности и выполнив над расширенной матрицей преобразованиее Гаусса-Жордана. $(A|E) \rightarrow (E|A^{-1})$

Протетстировать обратную мтарицу, которая сформируется на месте добавленной единичной матрицы, затем найти решение через эту обратную матрицу. ($x = A^{-1}b$)

Далее протестировать полученное решение, найдя для него невзяку $dx = b - Ax$.

Теория всё та же, что и с методом Гаусса, что очень тривиальна в доказательстве. Потому здесь я даже для себя расписывать ничего не буду. Реализация метода абсолютно такая же, если не проще. Асимптотика решения остаётся всё той же $O(N^3)$.

Зато я научусь находить обратную матрицу самостоятельно. Теперь я точно хочу реализовать хотя бы метод гаусса на плюсах.

In [88]:
import numpy as np
from typing import Tuple
from sympy import Matrix, init_printing
init_printing()

def isEqual(x, y) -> bool:
  return np.isclose(x, y, rtol=1e-05, atol=1e-08, equal_nan=False)

def GenerateMatrixOnes(N: int) -> Tuple[np.ndarray, np.ndarray]:
  Matr = np.full((N, N), 0.5) - np.random.rand(N, N)
  A = np.dot(Matr.T, Matr)
  b = np.zeros((N, 1))
  for i in range(N):
    b[i] += np.sum(A[i, :])

  return (A, b)

def GaussianElimination(A: np.ndarray) -> Tuple[np.ndarray, int]:
  C = A.copy()
  rank: int = 1

  for i in range(0, N - 1):
    # если на гл. диагонали возник ноль, то поднимаем наверх первую строку
    # с ненулевым значением на i-том столбце
    if np.abs(C[i, i]) == 0:
      for k in range(i + 1, N):
        if np.abs(C[k, i]) > np.abs(C[i, i]):
          C[[i, k]] = C[[k, i]]
          break

    # если же весь нижний столбец нулевой, то пропускаем шаг
    if (isEqual(C[i, i], 0)):
      continue

    rank += 1
    # сам метод Гаусса
    for j in range(i + 1, N):
      m = C[j, i] / C[i, i]
      C[j, : ] = C[j, : ] - m * C[i, : ]

  return (C, rank)

def StepBack(C: np.ndarray) -> None:
  N = C.shape[0]

  for i in range(N - 1, -1, -1):
    C[i] /= C[i, i]
    for j in range(i - 1, -1, -1):
      C[j] -= C[j, i] * C[i]

In [92]:
N: int = 3

A, b = GenerateMatrixOnes(N)
E = np.eye(N)
print("Сгенерированная матрица")
display(Matrix(A))

C = A.copy()
expanded = np.concatenate([C, E], axis=1)
print("Расширенная матрица")
display(Matrix(expanded))

GE_exp, rank = GaussianElimination(expanded)

print(f"Ранг матрицы = {rank}")
if rank == N:
  StepBack(GE_exp)

invA = np.split(GE_exp, 2, axis=1)[1]

x = np.matmul(invA, b)
dx = b - np.matmul(A, x)


print("Расширенная матрица после метода Гаусса")
display(Matrix(GE_exp))
print("Обратная матрица")
display(Matrix(invA))
print("Обратная матрица функцией invert")
display(Matrix(np.linalg.inv(A)))
print("Проверка А * А^-1")
display(Matrix(np.matmul(A, invA)))
print("Подсчет невязки")
display(Matrix(dx))

Сгенерированная матрица


⎡0.348034424333788   0.344606206858123   -0.139039593924358⎤
⎢                                                          ⎥
⎢0.344606206858123   0.401649174464107   -0.263707732165199⎥
⎢                                                          ⎥
⎣-0.139039593924358  -0.263707732165199  0.347692660611049 ⎦

Расширенная матрица


⎡0.348034424333788   0.344606206858123   -0.139039593924358  1.0   0    0 ⎤
⎢                                                                         ⎥
⎢0.344606206858123   0.401649174464107   -0.263707732165199   0   1.0   0 ⎥
⎢                                                                         ⎥
⎣-0.139039593924358  -0.263707732165199  0.347692660611049    0    0   1.0⎦

Ранг матрицы = 3
Расширенная матрица после метода Гаусса


...

⎡113.740218150256   -134.899649971822  -56.8309005566649⎤
⎢                                                       ⎥
⎢-134.899649971822   164.95477068625   71.1647346578908 ⎥
⎢                                                       ⎥
⎣-56.8309005566649  71.1647346578908   34.1248084735649 ⎦

Проверка А * А^-1


⎡         1.0           1.18970537167505e-15  -5.08953131499752e-16⎤
⎢                                                                  ⎥
⎢-1.4763856555471e-15     1.00000000000001    -2.91108371275916e-15⎥
⎢                                                                  ⎥
⎣-1.51446665482957e-15  4.82608151584318e-15           1.0         ⎦

Подсчет невязки


⎡-6.66133814775094e-16⎤
⎢                     ⎥
⎢-9.43689570931383e-16⎥
⎢                     ⎥
⎣6.86950496486816e-16 ⎦